# Hyperparameter grid search
NB the input data to the DNN is not normalised.

Hyperparameter grid search adapted from Machine Learning Mastery
https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
Using scikit-learn to grid search the batch size and epochs

In [98]:
import sys
from pathlib import Path
from datetime import datetime
from dateutil.tz import gettz

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras import utils
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV

np.random.seed(757566)

# User inputs

In [99]:
fname = 'private_dog0_correct_plus' # private_dog0_correct_plus

logs_dir = '../logs'
timestamp = '{:%Y-%m-%dT%H:%M}'.format(datetime.now(gettz("Europe/London")))
logs_dir = logs_dir +'/' + timestamp
if 'private' in fname:
    fdir = '../data/private_data/private_events_dev2' 
else:
    fdir = '../data' 

# Utilities

In [100]:
def readucr(filename):
    ''' Load a dataset from a file in UCR format
    space delimited, class labels in the first column.
    Returns
    X : DNN input data
    Y : class labels
    '''
    data = np.loadtxt(Path(filename))
    Y = data[:,0]
    X = data[:,1:]
    return X, Y


def prepare_data(y):
    ''' Return y as a categorical array'''
    nb_classes = 2
    y = (y - y.min())/(y.max()-y.min())*(nb_classes-1)
    Y = utils.to_categorical(y, nb_classes)
    return Y

# Create model

In [101]:
def create_resnet(input_shape=(0,0), num_features0=-1):
    ''' Return ResNet model '''
    nb_classes = 2
    filter_size = 4
    #num_features0 = 64
    num_features1 = 64
    num_features2 = 128
    identity_shortcut = True
    
    # First block
    x = Input(shape=(input_shape))
    conv = x
    skip = x
    conv = keras.layers.Conv1D(num_features0, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    
    conv = keras.layers.Conv1D(num_features1, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    
    conv = keras.layers.Conv1D(num_features2, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    shortcut = keras.layers.Conv1D(num_features2, filter_size, padding='same')(skip)
    shortcut = keras.layers.BatchNormalization()(shortcut)
    conv = keras.layers.add([conv, shortcut])
    conv = Activation('relu')(conv)
    
    # Second block
    skip = conv
    conv = keras.layers.Conv1D(num_features0, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    
    conv = keras.layers.Conv1D(num_features1, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    
    conv = keras.layers.Conv1D(num_features2, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    if identity_shortcut:
        shortcut = skip
    else:
        shortcut = keras.layers.Conv1D(num_features2, filter_size, padding='same')(skip)
        shortcut = keras.layers.BatchNormalization()(shortcut)
    conv = keras.layers.add([conv, shortcut])
    conv = Activation('relu')(conv)
    
    # Third block
    skip = conv
    conv = keras.layers.Conv1D(num_features0*2, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    
    conv = keras.layers.Conv1D(num_features1*2, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    
    conv = keras.layers.Conv1D(num_features2*2, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    shortcut = keras.layers.Conv1D(num_features2*2, filter_size, padding='same')(skip)
    shortcut = keras.layers.BatchNormalization()(shortcut)
    conv = keras.layers.add([conv, shortcut])
    conv = Activation('relu')(conv)
    
    # Output block
    full = keras.layers.GlobalAveragePooling1D()(conv)   
    out = Dense(nb_classes, activation='sigmoid')(full)
    model = Model(x, out)
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model
    

# Run

In [102]:
# load dataset
x_train, y_train = readucr(fdir+'/'+fname+'/'+fname+'_TRAIN.txt')
x_test, y_test = readucr(fdir+'/'+fname+'/'+fname+'_TEST.txt')
X = np.concatenate((x_train, x_test), axis=0)
Y = np.concatenate((y_train, y_test), axis=0)
X = X.reshape(X.shape + (1,))
input_shape = X.shape[1:]
print(input_shape)
Y = prepare_data(Y)

# define the grid search parameters
batch_size = 32
epochs = 1000
num_features0 = [64]
num_features1 = 64 #[64, 128, 256]
filter_size = 4
pooling_size = 4
param_grid = dict(num_features0=num_features0)

# Create model and run the grid search
if False:
    model = KerasClassifier(build_fn=create_resnet, 
                            input_shape=input_shape,
                            #num_features0=num_features0, filter_size=filter_size, pooling_size=pooling_size,
                            batch_size=batch_size, epochs=epochs,
                            verbose=1)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, error_score=0, return_train_score=True)
    grid_result = grid.fit(X, Y)

    # Summarise results
    print('Best score:', grid_result.best_score_, 'using: ', grid_result.best_params_)
    cv = pd.DataFrame(grid_result.cv_results_)
    pd.set_option('display.max_colwidth', -1)
    cv[['mean_test_score', 'std_test_score', 'params']]
else:
    x_train = x_train.reshape(x_train.shape + (1,))
    x_test = x_test.reshape(x_test.shape + (1,))
    Y_train = prepare_data(y_train)
    Y_test = prepare_data(y_test)

    callbacks = []
    tensorboard_dir = '../logs/tensorboard'
    tensorboard_dir = tensorboard_dir +'/' + timestamp
    tb_dir = tensorboard_dir+'/'+fname
    Path(tb_dir).mkdir(parents=True, exist_ok=True) 
    callbacks.append(keras.callbacks.TensorBoard(log_dir=tb_dir, histogram_freq=0))
    model = create_resnet(input_shape, num_features0[0])
    hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=epochs,
                      verbose=1, validation_data=(x_test, Y_test), callbacks=callbacks)
       

(1000, 1)
Train on 76 samples, validate on 26 samples
Epoch 1/1000
76/76 [==============================] - 6s 73ms/sample - loss: 0.6349 - acc: 0.5855 - val_loss: 0.6144 - val_acc: 0.6346
Epoch 2/1000
76/76 [==============================] - 0s 3ms/sample - loss: 0.3924 - acc: 0.7697 - val_loss: 0.5862 - val_acc: 0.8077
Epoch 3/1000
76/76 [==============================] - 0s 3ms/sample - loss: 0.3938 - acc: 0.8026 - val_loss: 0.5631 - val_acc: 0.8462
Epoch 4/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.3627 - acc: 0.8355 - val_loss: 0.5365 - val_acc: 0.8462
Epoch 5/1000
76/76 [==============================] - 0s 3ms/sample - loss: 0.3286 - acc: 0.8553 - val_loss: 0.5224 - val_acc: 0.8462
Epoch 6/1000
76/76 [==============================] - 0s 3ms/sample - loss: 0.3796 - acc: 0.7895 - val_loss: 0.5182 - val_acc: 0.8846
Epoch 7/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.3475 - acc: 0.8618 - val_loss: 0.5073 - val_acc: 0.8846
Epoch 8

76/76 [==============================] - 0s 2ms/sample - loss: 0.1480 - acc: 0.9737 - val_loss: 0.6073 - val_acc: 0.5000
Epoch 62/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.2327 - acc: 0.9079 - val_loss: 0.4852 - val_acc: 0.7500
Epoch 63/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.1627 - acc: 0.9605 - val_loss: 1.1720 - val_acc: 0.5000
Epoch 64/1000
76/76 [==============================] - 0s 3ms/sample - loss: 0.2362 - acc: 0.9474 - val_loss: 1.4789 - val_acc: 0.1731
Epoch 65/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.1693 - acc: 0.9342 - val_loss: 4.6785 - val_acc: 0.4808
Epoch 66/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.3215 - acc: 0.8553 - val_loss: 6.8557 - val_acc: 0.5000
Epoch 67/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.3121 - acc: 0.8684 - val_loss: 5.4045 - val_acc: 0.5000
Epoch 68/1000
76/76 [==============================] - 0s 2ms/sample 

76/76 [==============================] - 0s 2ms/sample - loss: 0.0369 - acc: 1.0000 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 122/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0375 - acc: 1.0000 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 123/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0372 - acc: 1.0000 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 124/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.1087 - acc: 0.9605 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 125/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.2371 - acc: 0.8947 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 126/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.1748 - acc: 0.9342 - val_loss: 7.9919 - val_acc: 0.5000
Epoch 127/1000
76/76 [==============================] - 0s 3ms/sample - loss: 0.1301 - acc: 0.9342 - val_loss: 7.7048 - val_acc: 0.5000
Epoch 128/1000
76/76 [==============================] - 0s 2ms/

76/76 [==============================] - 0s 3ms/sample - loss: 0.0206 - acc: 1.0000 - val_loss: 0.7036 - val_acc: 0.6538
Epoch 182/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0232 - acc: 1.0000 - val_loss: 0.9094 - val_acc: 0.5577
Epoch 183/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0188 - acc: 1.0000 - val_loss: 1.4554 - val_acc: 0.5000
Epoch 184/1000
76/76 [==============================] - 0s 3ms/sample - loss: 0.0155 - acc: 1.0000 - val_loss: 2.5283 - val_acc: 0.5000
Epoch 185/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0135 - acc: 1.0000 - val_loss: 3.4384 - val_acc: 0.5000
Epoch 186/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0194 - acc: 1.0000 - val_loss: 4.9975 - val_acc: 0.5000
Epoch 187/1000
76/76 [==============================] - 0s 3ms/sample - loss: 0.0162 - acc: 1.0000 - val_loss: 4.5021 - val_acc: 0.5000
Epoch 188/1000
76/76 [==============================] - 0s 3ms/

76/76 [==============================] - 0s 3ms/sample - loss: 0.0084 - acc: 1.0000 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 242/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0140 - acc: 1.0000 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 243/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0272 - acc: 0.9868 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 244/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0082 - acc: 1.0000 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 245/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0130 - acc: 1.0000 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 246/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0084 - acc: 1.0000 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 247/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0074 - acc: 1.0000 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 248/1000
76/76 [==============================] - 0s 2ms/

...

76/76 [==============================] - 0s 2ms/sample - loss: 0.0074 - acc: 1.0000 - val_loss: 0.9336 - val_acc: 0.8462
Epoch 719/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0082 - acc: 1.0000 - val_loss: 1.0546 - val_acc: 0.8462
Epoch 720/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0081 - acc: 1.0000 - val_loss: 1.3625 - val_acc: 0.8077
Epoch 721/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0072 - acc: 1.0000 - val_loss: 1.4642 - val_acc: 0.8077
Epoch 722/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0084 - acc: 1.0000 - val_loss: 1.4447 - val_acc: 0.8077
Epoch 723/1000
76/76 [==============================] - 0s 3ms/sample - loss: 0.0063 - acc: 1.0000 - val_loss: 1.1853 - val_acc: 0.8077
Epoch 724/1000
76/76 [==============================] - 0s 3ms/sample - loss: 0.0043 - acc: 1.0000 - val_loss: 0.9013 - val_acc: 0.8462
Epoch 725/1000
76/76 [==============================] - 0s 2ms/

76/76 [==============================] - 0s 2ms/sample - loss: 0.0031 - acc: 1.0000 - val_loss: 0.5722 - val_acc: 0.8462
Epoch 779/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0021 - acc: 1.0000 - val_loss: 0.5351 - val_acc: 0.8462
Epoch 780/1000
76/76 [==============================] - 0s 3ms/sample - loss: 0.0019 - acc: 1.0000 - val_loss: 0.5050 - val_acc: 0.8462
Epoch 781/1000
76/76 [==============================] - 0s 3ms/sample - loss: 0.0024 - acc: 1.0000 - val_loss: 0.4823 - val_acc: 0.8077
Epoch 782/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0017 - acc: 1.0000 - val_loss: 0.4669 - val_acc: 0.8077
Epoch 783/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0026 - acc: 1.0000 - val_loss: 0.4549 - val_acc: 0.8077
Epoch 784/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0058 - acc: 1.0000 - val_loss: 0.4628 - val_acc: 0.8462
Epoch 785/1000
76/76 [==============================] - 0s 2ms/

76/76 [==============================] - 0s 2ms/sample - loss: 0.0019 - acc: 1.0000 - val_loss: 0.5418 - val_acc: 0.8462
Epoch 839/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0027 - acc: 1.0000 - val_loss: 0.5425 - val_acc: 0.8462
Epoch 840/1000
76/76 [==============================] - 0s 3ms/sample - loss: 0.0017 - acc: 1.0000 - val_loss: 0.5542 - val_acc: 0.8462
Epoch 841/1000
76/76 [==============================] - 0s 3ms/sample - loss: 0.0015 - acc: 1.0000 - val_loss: 0.5649 - val_acc: 0.8462
Epoch 842/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0222 - acc: 0.9868 - val_loss: 0.5271 - val_acc: 0.8462
Epoch 843/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0055 - acc: 1.0000 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 844/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0437 - acc: 0.9868 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 845/1000
76/76 [==============================] - 0s 2ms/

76/76 [==============================] - 0s 2ms/sample - loss: 0.0027 - acc: 1.0000 - val_loss: 0.7498 - val_acc: 0.8462
Epoch 899/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0032 - acc: 1.0000 - val_loss: 0.9122 - val_acc: 0.8077
Epoch 900/1000
76/76 [==============================] - 0s 3ms/sample - loss: 0.0017 - acc: 1.0000 - val_loss: 1.0440 - val_acc: 0.8077
Epoch 901/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0018 - acc: 1.0000 - val_loss: 1.1012 - val_acc: 0.7308
Epoch 902/1000
76/76 [==============================] - 0s 2ms/sample - loss: 8.8723e-04 - acc: 1.0000 - val_loss: 1.0538 - val_acc: 0.7308
Epoch 903/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0015 - acc: 1.0000 - val_loss: 1.0242 - val_acc: 0.7500
Epoch 904/1000
76/76 [==============================] - 0s 2ms/sample - loss: 8.0189e-04 - acc: 1.0000 - val_loss: 1.0117 - val_acc: 0.7692
Epoch 905/1000
76/76 [==============================] -

76/76 [==============================] - 0s 3ms/sample - loss: 5.6521e-04 - acc: 1.0000 - val_loss: 0.5810 - val_acc: 0.8846
Epoch 958/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0016 - acc: 1.0000 - val_loss: 0.5884 - val_acc: 0.8846
Epoch 959/1000
76/76 [==============================] - 0s 2ms/sample - loss: 8.4693e-04 - acc: 1.0000 - val_loss: 0.6153 - val_acc: 0.8846
Epoch 960/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0010 - acc: 1.0000 - val_loss: 0.6625 - val_acc: 0.8846
Epoch 961/1000
76/76 [==============================] - 0s 2ms/sample - loss: 0.0017 - acc: 1.0000 - val_loss: 0.7012 - val_acc: 0.8462
Epoch 962/1000
76/76 [==============================] - 0s 2ms/sample - loss: 5.8814e-04 - acc: 1.0000 - val_loss: 0.7226 - val_acc: 0.8462
Epoch 963/1000
76/76 [==============================] - 0s 2ms/sample - loss: 7.6656e-04 - acc: 1.0000 - val_loss: 0.7372 - val_acc: 0.8462
Epoch 964/1000
76/76 [=========================

In [103]:
cv

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_num_features0,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,201.757038,8.289066,0.480392,0.47549,64,{'num_features0': 64},1,0.441176,0.389706,0.470588,0.514706,0.529412,0.522059,2.093201,0.585837,0.036683,0.060733


In [104]:
print('Completed at', '{:%Y-%m-%dT%H:%M}'.format(datetime.now(gettz("Europe/London"))))

Completed at 2019-04-13T19:51


In [105]:
Path(logs_dir+'/'+fname).mkdir(parents=True, exist_ok=True)
filename = Path(logs_dir+'/'+fname+'/resnet_grid_search_summary.csv')
cv.to_csv(filename)
print('Results saved to', filename)

Results saved to ../logs/2019-04-13T19:47/private_dog0_correct_plus/resnet_grid_search_summary.csv
